In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir('../')

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
data = pd.read_csv("./datasets/Iris.csv")
data["Species"] = data["Species"].apply(lambda x: 1 if x=="Iris-setosa" else 0)
data = data.drop(["Id"],axis=1)
data = data.sample(frac=1).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [5]:
x = data.drop(["Species"],axis=1) #input data
y = data["Species"] #ground truth
X = x.to_numpy().astype(np.float64)
Y = y.to_numpy().astype(np.float64)

In [6]:
x_train,x_test = X[:100],X[100:]
y_train,y_test = Y[:100],Y[100:]

Sigmoid Function
$$
\sigma = \frac{1}{1 + e^{-z}}
$$
$$
\hat{y} = \frac{1}{1 + e^{-(m*x+c)}}
$$
Binary Cross Entropy (cost function)
$$
\hat{y} = - \sum \left( y \log(\hat{y}) + (1 - y) \log(1 - \hat{y}) \right)
$$
partial derivative of cost function
$$
\frac{\partial \hat{y}}{\partial \mathbf{w}} = - \sum   \frac{y}{\hat{y}} \frac{\partial \hat{y}}{\partial \mathbf{w}} - \frac{1 - y}{1 - \hat{y}} \frac{\partial \hat{y}}{\partial \mathbf{w}}
$$
deravative of sigmoid function
$$
\frac{\partial \sigma}{\partial \mathbf{Z}} = - \sum   {y}{\left( 1 - \hat y \right)} -{\left( 1 - y \right)\hat y}
$$

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def predict(w,b,X):
    z=0
    for i in range(X.shape[1]):
        z += w[i] * X[:,i]
    z = np.array(b+z)
    # print(z)
    return sigmoid(z)
def cost_function(w,b,X,Y):
    y_pred = predict(w,b,X)
    print(min(y_pred),max(y_pred))
    # print(y_pred)
    return -1 * sum(Y*np.log(y_pred) + (1-Y)*np.log((1-y_pred)))
def gradient(w,b,X,Y):
    y_pred = predict(w,b,X)
    # print(y_pred)
    new_b = -1 * sum(Y*(1-y_pred) - (1-Y)*y_pred)
    new_w = np.zeros(w.shape[0])
    for i in range(w.shape[0]):
        new_w[i] = -1 * sum(Y*(1-y_pred)*X[:,i] - (1-Y)*y_pred*X[:,i])
    return new_w,new_b
epochs=1000
def descent(w,b,X,Y,lr):
    for i in range(epochs):
        # print(i,b)
        
        new_w,new_b = gradient(w,b,X,Y)
        b = b - lr * new_b
        w = w - lr * new_w
        # print(b,w)
        # print("#cost",i,cost_function(w,b,X,Y))
    return w,b
w = np.ones(X.shape[1])
b=1
new_w,new_b = descent(w,b,x_train,y_train,0.001)
print(new_w)

[ 0.15943215  2.09097499 -2.9172241  -0.64268761]


In [8]:
def calc(X):
    temp =  np.dot(X,new_w)+new_b
    return [1 if i>=0.5 else 0 for i in temp]

In [9]:
def accuracy(Y,y_pred):
    return sum(Y==y_pred)/len(Y)

In [10]:
print(accuracy(y_test,calc(x_test)))

1.0


In [14]:
data = pd.read_csv("./datasets/mushrooms.csv")
data = data.sample(frac=1).reset_index(drop=True)
x = data.drop(["class"],axis=1) #input data
y = data["class"] #ground truth (0 = edible or 1 = poisonous)

from utils.LabelEncoder import LabelEncoder
transformed_x = pd.DataFrame()
for c in x:
    le =  LabelEncoder(1)
    transformed_x[c] =  le.fit_transform(x[c])

from utils.StandardScaler import StandardScaler
sc = StandardScaler()
scaled_x = sc.transform(transformed_x)

X = scaled_x.to_numpy().astype(np.int64)
Y = np.array([0 if i=="e" else 1 for i in y]).astype(np.int64)
data.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,b,t,f,f,c,b,h,...,f,w,w,p,w,o,p,h,v,u
1,e,f,y,g,t,n,f,c,b,w,...,s,g,p,p,w,o,p,k,v,d
2,p,x,y,g,f,f,f,c,b,h,...,k,p,p,p,w,o,l,h,v,g
3,p,x,y,n,t,p,f,c,n,w,...,s,w,w,p,w,o,p,k,s,g
4,e,f,y,n,t,n,f,c,b,n,...,s,w,g,p,w,o,p,k,y,d


In [15]:
x_train,x_test = X[:7500],X[7500:]
y_train,y_test = Y[:7500],Y[7500:]

In [18]:
w = np.zeros(X.shape[1])
b=0
new_w,new_b = descent(w,b,x_train,y_train,0.001)
print(new_w)
print(accuracy(y_test,calc(x_test)))

[-0.50275466 -2.68090689  0.0129549   3.99942259  6.51805656 -1.42759955
 -4.36745057  2.13190452 -0.21262614  0.4013152  -9.02805745 -0.73660097
  2.30824018  0.59018218  0.35957306  1.00631193  0.31285599  0.32717619
 -1.13868977 -0.24931883 -1.4873855  -0.24485226]
0.9342948717948718
